# Clean up my Trip List

**Now obsolete** The source files have been changed.

Clean up the trip list I've kept in an excell file.

* Load the file.
* Normalize
* Substitute actual people's names
* Load recent mountaineer trips
* Merge
* Write to a CSV

Then hand edit to clean up duplicates

In [ ]:
import pandas as pd
import seaborn as sns
from pathlib import Path
import trips_mtn

TRIPS_DIR = Path(Path.home(), "Documents/trips")
TRIPS_FILE = Path(TRIPS_DIR, "TripList.xlsx")
TRIPS_FILE.exists()

In [ ]:
# Load trips flie
trips = pd.read_excel(TRIPS_FILE)
trips.info()

In [ ]:
# Normalize trip types and sub-type.

# Split "type" into two components.
triptypedf = trips['Type'].str.split("/", n=2, expand=True)

# Normalize the first component and make that the new Types column
trips['Type'] = triptypedf[0].str.capitalize()


In [ ]:
#
# Joining sub type is harder.  Some sub-types come from the type and some are already in the field.
#
# Get subtype that was part of type.
a = triptypedf[1].fillna('').str.capitalize()

def either(a,b):
	'''Return either a or b, which ever one has a value.  Or the combined value if both have value.'''
	if a and b:
		return str(a) + "|" + str(b)
	if a:
		return a
	if b:
		return b
	return ""

# Combine two series.
z = a.combine(trips['SubType'].fillna("").str.capitalize(), either, fill_value="")
trips['SubType'] = z


In [ ]:
#
# Load the trip partners table.  From that create a new dataframe that maps from codes used to person names.
# 
partnersdf = pd.read_excel(TRIPS_FILE, sheet_name="TripPartners")
fullname = partnersdf['First'] + ' ' + partnersdf['Last']
code = partnersdf['Code']
namemapdf = pd.DataFrame(data = {"code": code, "name": fullname})
namemapdf.set_index(keys='code', inplace=True)
#namemapdf.head()


In [ ]:
#
# use the partners mapping to replace codes with full names
#
for i in trips.index:
	for col in ('Ldr', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6'):
		if pd.isna(trips.iloc[i][col]):
			trips.at[i, col] = ""
		else:
			v = trips.iloc[i][col]
			if v:
				try:
					# Get the full name from the map
					fullname = namemapdf.at[v.lower(), 'name']
					trips.at[i, col] = fullname
				except:
					# If look up did not work, use the value tha twas there.
					pass

In [ ]:
trips.columns

In [ ]:
mtndf = trips_mtn.load_mtn_activity()
mtndf.to_csv("local-mtndf.csv")
desiredColumnOrder = list(mtndf.columns)
#desiredColumnOrder

In [ ]:
trips['Mtn'] = ''			# Add field indicating this is not a mountainer trip
trips = trips[desiredColumnOrder]
trips = trips.append(mtndf, ignore_index=True)
trips.sort_values(by='Date', inplace=True)

In [ ]:
trips.columns


In [ ]:
trips.to_csv("local-trips.csv", date_format="%Y-%m-%d", index=False)

In [ ]:
trips

In [ ]:
trips['Type'].unique()

# Create trip partner list
Maybe read from second sheet
